# ChiselDB：调试友好的结构化数据库

**ChiselDB：Debug-friendly structured database**

**动机**

- 波形十分巨大并且难以进行分析

- 我们想要分析一些类似访存事务 trace 的结构化数据


**Motivation**

- Waveforms are large in size and hard to apply further analysis

- Need to analyze structured data like memory transaction trace

我们提出了 ChiselDB，用于存储结构化数据辅助快速定位问题。

**We propose ChiselDB for storage of structured data to support faster bug localization.**

**优势**

- 在硬件模块的接口处插入探针

- 使用DPI-C从Chisel中传输数据

- 支持数据库操作例如：SQL

**Highlights**

- Inserting probes between module interfaces in hardware

- DPI-C: Using C++ function in Chisel code to transfer data

- Persist in database, SQL queries supported

<div align="center">
  <img src="../images/02-functional/04-chiseldb/chiseldb-en.png" alt="chiselDB-overview" style="width: 30%; height: auto;">
</div>


ChiselDB 是一个用于辅助功能/性能调试的结构化数据库。它使用 DPI-C 接口将 Chisel Bundle 的数据记录到 sqlite 数据库中。

ChiselDB is a structured database for aiding functional and performance debugging. It uses a DPI-C interface to log Chisel Bundle data into an SQLite database.

相比 verilog wire，Chisel Bundle 具有更高层的语义信息（可以类比成数个 verilog wire 组成的组），后续使用 SQL 查询语句访问这些结构化的数据进行分析。

Compared to Verilog wires, Chisel Bundles carry higher-level semantics—essentially groups of multiple wires. You can then access these structured data with SQL queries for analysis.

我们提供了一个预先编译的 `emu-cdb-err` 仿真程序，其存在一个人为注入的 bug，将从 L2 Cache 到 L3 Cache 释放数据都置为常量值。

We provide a prebuilt simulator `emu-cdb-err` with an injected bug that forces all data released from L2 Cache to L3 Cache to a constant value.

通过 `--dump-db` 参数启用 ChiselDB，同时启用 difftest，运行后 difftest 报错，并在 `./build` 路径下生成了 `.db` 文件。

Enable ChiselDB with `--dump-db` and turn on DiffTest; after running, DiffTest reports an error and a `.db` file is generated under `./build`.

In [ ]:
%%bash
cd .. && source env.sh
cd ${NOOP_HOME}

rm -f ./build/*.db # clean old files
mkdir -p build

$(get_asset emu-precompile/emu-cdb-err) \
    -i $(get_asset workload/stream_100000.bin) \
    --diff $(get_asset emu-precompile/riscv64-nemu-interpreter-so) \
    --dump-db \
    2>linux.err || true

echo -n "Dump DB: "
realpath ./build/*.db

随后我们可以使用 sqlite 读取 db 文件并进行分析，下面查询所有地址为 `0x80048f00` 的 tilelink transaction，并使用 `./scripts/cache/convert_tllog.sh` 脚本来格式化输出。

Then use SQLite to read the `.db` for analysis: query all TileLink transactions at address `0x800419c0`, and format the output with `./scripts/cache/convert_tllog.sh`.

In [ ]:
%%bash
cd .. && source env.sh

DB=$(ls -t ${NOOP_HOME}/build/*db | head -n 1)

sqlite3 ${DB} "select * from TLLog where ADDRESS=0x80048f00" | sh ${NOOP_HOME}/scripts/cache/convert_tllog.sh

Result: [Time/To_From/Channel/Opcode/Permission/Address/Data]

**Data successfully transferred from L1D to L2**

16171 L2_L1D_0 C ProbeAckData Shrink TtoN 0 5 80048f00 0000000080048f50 0000000080014328 0000000000000000 0000000000000000 user: 0 echo: 0 

16172 L2_L1D_0 C ProbeAckData Shrink TtoN 0 5 80048f00 0000000000000000 0000000000000000 000000008001e000 0000000080042060 user: 0 echo: 0 



**Data successfully transferred from L2 to L3**

16179 L3_L2_0 C ProbeAckData Shrink TtoN 0 2 80048f00 0000000080048f50 0000000080014328 0000000000000000 0000000000000000 user: 0 echo: 1 

16180 L3_L2_0 C ProbeAckData Shrink TtoN 0 2 80048f00 0000000000000000 0000000000000000 000000008001e000 0000000080042060 user: 0 echo: 1

**But when L1D acquires Eaddr again, data loaded from L3 is wrong**

16457 L2_L1D_0 A AcquireBlock Grow NtoT 0 0 80048f00 0000000000000000 0000000000000000 0000000000000000 0000000000000000 user: 80048f07 echo: 0 

16463 L3_L2_0 A AcquireBlock Grow NtoT 0 0 80048f00 0000000000000000 0000000000000000 0000000000000000 0000000000000000 user: 0 echo: 1 

16486 L3_L2_0 D GrantData Cap toT 1 0 80048f00 **0000000000abcdef** 0000000000000000 0000000000000000 0000000000000000 user: 0 echo: 1 

**So there must be something wrong when L3 records Release Data**